# EDA on textual data

## Import libraries

In [765]:
import pathlib as path
from tqdm import tqdm

import pandas as pd
from collections import  Counter

import spacy
from cleantext import clean
import re

import plotly.express as px
from IPython.display import display,Markdown

## Define constants

In [766]:
FOLDER  = path.Path("/home/jovyan/data/")

SRC_FILE_NAME = "covid19db.json"

OUTPUT_FILE_NAME = "eda_pwdb_result.html"

SRC_FILE_PATH = FOLDER / SRC_FILE_NAME

OUTPUT_FILE_PATH = FOLDER / OUTPUT_FILE_NAME

TEXTUAL_COLUMNS = ['title','background_info_description','content_of_measure_description','use_of_measure_description','involvement_of_social_partners_description']


## Load PWDB dataset & SpaCy NLP component

In [767]:
df = None
if SRC_FILE_PATH.exists():
    df = pd.read_json(SRC_FILE_PATH)
    nlp = spacy.load("en_core_web_sm",exclude=["lemmatizer"])
else:
    print("Input source file is invalid!")

## Functions to calculate frequency for textual data

In [768]:
def calculate_frequency(data : pd.Series,title : str,relative = False):
    observation_type_name = 'Absolute freq' if not relative else 'Relative freq'
    data.dropna(inplace=True)
    observation = pd.DataFrame(Counter(data).most_common(10),columns=[title,observation_type_name])
    if relative:
        observation[observation_type_name]/=observation[observation_type_name].sum()/100
        observation[observation_type_name]=round(observation[observation_type_name],2)
    return observation

## Function to get spaCy NLP doc for each row from series of strings

In [769]:
def get_nlp_docs(data : pd.Series):
    return [nlp(row) for row in data]

## Function to get entity names from series of strings

In [770]:
def get_named_entities(data: pd.Series,docs : list = None):
    if docs is None:
        docs = get_nlp_docs(data)
    result = [e.label_ for doc in docs for e in doc.ents]
    return pd.Series(result,dtype=str)

## Function to get list of words labeled with entity_name

In [771]:
def get_entity_words(data : pd.Series, entity_name : str = 'ORG',docs : list = None):
    if docs is None:
        docs = get_nlp_docs(data)
    result = [e.text for doc in docs for e in doc.ents if e.label_== entity_name]
    return pd.Series(result,dtype=str)

## Function which remove stop words from series of strings

In [772]:
def remove_stopwords(data : pd.Series):
    stop_words = nlp.Defaults.stop_words
    result = []
    for row in data:
        result.append(" ".join([word for word in row.split() if word not in stop_words]))
    return pd.Series(result,dtype=str)

## Function to get N grams from series of strings

In [773]:
def get_ngrams(data : pd.Series, n : int,stopwords : bool = True):
    if not stopwords:
        data = remove_stopwords(data)
    result = [ " ".join(text[i:i+n])
               for text in data.str.split()
               for i in range(len(text)-n+1)]
    return pd.Series(result,dtype=str)

## Function to get list of noun phrases

In [774]:
def get_noun_phrases(data : pd.Series,docs : list = None):
    if docs is None:
        docs = get_nlp_docs(data)
    result = [str(n) for doc in docs for n in doc.noun_chunks ]
    return pd.Series(result,dtype=str)

## Function to get list of words without stop words

In [775]:
def get_words(data : pd.Series):
    text = " ".join(data)
    stop_words = nlp.Defaults.stop_words
    result = [ word for word in text.split() if word not in stop_words]
    return pd.Series(result,dtype=str)

## Function to delete punctuation form text

In [776]:
def delete_punctuation(text : str):
    regex_filter = r'[,;:\*`#\'\"^&~@=+_.()?\[\]!\s]\s*'
    text = " ".join(list(filter(None,re.split(regex_filter,text))))
    return text

## Function to clear textual data
- remove stop-words
- transform text to lowercase
- remove textual "artifacts"
- remove emails
- remove urls
- remove phone numbers


In [777]:
def prepare_text_data(data : pd.Series ):
    data = data.dropna().explode()
    result = [ delete_punctuation(
                clean(text,
                     no_urls=True,
                     no_emails=True,
                     no_phone_numbers=True,
                     )
                )
              for text in data
             ]
    return pd.Series(result,dtype=str)

## Function for plot bar chart on observations

In [778]:
def plot_bar_chart(observations: pd.DataFrame,chart_title: str):
    columns = observations.columns
    return px.bar(observations,x=columns[1],y=columns[0],title=chart_title)

## Function for plot pie chart on observations

In [779]:
def plot_pie_chart(observations: pd.DataFrame,chart_title: str):
    columns = observations.columns
    return px.pie(observations,values=columns[1],names=columns[0],title=chart_title )

## Function to display result of textual EDA

In [780]:
def eda_display_result(result : pd.DataFrame,chart_title : str):
    if result.size > 0 :
        display(Markdown(chart_title))
        display(tuple(result))
        plot_bar_chart(result, chart_title).show()
        plot_pie_chart(result, chart_title).show()

## Textual EDA for words classified with the same entity name

In [781]:
def eda_entity_words( data : pd.Series,data_title : str, docs ):
    result = calculate_frequency(get_entity_words(data,'ORG',docs),data_title,True)
    eda_display_result(result,"Entity words for "+data_title)

## Textual EDA for words frequency

In [782]:
def eda_words_freq( data : pd.Series,data_title : str):
    result = calculate_frequency(get_words(data),data_title,True)
    eda_display_result(result,"Words frequency for "+data_title)


## Textual EDA for entity names

In [783]:
def eda_named_entities( data : pd.Series,data_title : str, docs ):
    result = calculate_frequency(get_named_entities(data,docs),data_title,True)
    eda_display_result(result,"Named entities for "+data_title)

## Textual EDA for noun phrases

In [784]:
def eda_noun_phrases( data : pd.Series,data_title : str, docs ):
    result = calculate_frequency(get_noun_phrases(data,docs),data_title,True)
    eda_display_result(result,"Noun phrases for "+data_title)

## Textual EDA for N grams

In [785]:
def eda_n_grams( data : pd.Series,data_title : str, n_grams):
    result = calculate_frequency(get_ngrams(data,n_grams),data_title,True)
    eda_display_result(result,"N grams for "+data_title)

## Textual EDA for N grams without stop words

In [786]:
def eda_n_grams_without_stopwords( data : pd.Series,data_title : str, n_grams):
    result = calculate_frequency(get_ngrams(data,n_grams,False),data_title,True)
    eda_display_result(result,"N grams without stopwords for "+data_title)

## Combined textual EDA

In [787]:
def eda_textual(data : pd.DataFrame):
    pbar = tqdm(data.columns)
    for column_name in pbar:
        pbar.set_description('Eda on textual data ['+column_name+']')
        column_data = prepare_text_data(data[column_name])
        docs = get_nlp_docs(column_data)
        eda_words_freq(column_data,column_name)
        eda_n_grams(column_data,column_name,3)
        eda_n_grams_without_stopwords(column_data,column_name,3)
        eda_noun_phrases(column_data,column_name,docs)
        eda_named_entities(column_data,column_name,docs)
        eda_entity_words(column_data,column_name,docs)

## Execute combined textual EDA on specific columns from PWDB dataset

In [788]:
eda_textual(df[TEXTUAL_COLUMNS])




Eda on textual data [involvement_of_social_partners_description]: 100%|██████████| 5/5 [00:57<00:00, 11.51s/it]


Words frequency for title

,title,Relative freq
0,covid-19,17.16
1,support,15.68
2,workers,9.38
3,social,8.92
4,temporary,8.81
5,measures,8.58
6,companies,8.47
7,working,8.12
8,agreement,7.55
9,employees,7.32


N grams for title

,title,Relative freq
0,the covid-19 pandemic,15.31
1,in case of,12.24
2,health and safety,11.22
3,people with disabilities,11.22
4,social security contributions,9.18
5,measures to support,9.18
6,of the covid-19,8.16
7,financial support for,8.16
8,affected by the,8.16
9,exceptional and temporary,7.14


...

,involvement_of_social_partners_description,Relative freq
0,social partners involved,24.95
1,involvement social partners,14.50
2,economic social council,13.01
3,social partners consulted,10.23
4,level social partners,7.46
5,standing committee social,6.61
6,committee social concertation,6.61
7,social partners informed,5.54
8,design implementation monitoring,5.54
9,peak-level social partners,5.54


Noun phrases for involvement_of_social_partners_description

,involvement_of_social_partners_description,Relative freq
0,social partners,18.55
1,the measure,13.78
2,the social partners,13.51
3,the government,12.28
4,it,8.46
5,no involvement,8.32
6,they,8.25
7,this measure,6.34
8,measures,5.80
9,no information,4.71


Named entities for involvement_of_social_partners_description

,involvement_of_social_partners_description,Relative freq
0,CARDINAL,32.81
1,DATE,30.28
2,ORDINAL,17.98
3,PERSON,15.46
4,PERCENT,1.26
5,NORP,0.79
6,GPE,0.63
7,TIME,0.47
8,LAW,0.32
